In [25]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

In [26]:
#Hallo Herr Zimmermann,

 

#vielen Dank für Ihre Nachricht.

#Ich habe mir das Notebook angeschaut und eine Lösung vorgeschlagen. Allerdings verstehe ich das Ziel der Methode nicht ganz, von daher müssten Sie einmal prüfen, ob ich es richtig interpretiert habe.

#Mein Vorschlag kann folgendes: für ein Fenster von z.B. 10 Nachrichten vorher oder nachher (Achtung: nachher ist für eine Prediction nicht verwendbar! Zukunftsinformation nennt man das) wird gezählt, wie oft die severity-Klassen 1, 2 und 3 vorkommen. Dies lege ich in drei Spalten ab: pre_no_srv_1, pre_no_srv_2, pre_no_srv_3

#Die vielen Spalten, die durch das „Shiften“ zustande kommen, können Sie auch wieder löschen. Diese sind nur für den Zwischenschritt nötig.

#So bewege ich mich dauerhaft in der Pandas-Welt und von der Performance (Laufzeit und Speicher) gibt es keine Problem.

 

#Ist es das, was Sie erreichen wollten?

 

#Hinsichtlich der Mappings habe ich Ihnen auch noch einen Vorschlag gemacht.

 

#Viele Grüße und ein schönes Wochenende

#Peer Küppers 

In [28]:
#Daten einlesen
df_orig = pd.read_parquet("../data_source/event_ano.parquet.gzip")

SyntaxError: invalid syntax (<ipython-input-28-9bae51b47e44>, line 2)

In [4]:
df = df_orig.set_index("event_timestamp")
del df['Unnamed: 0']
df.sort_index(inplace=True)

In [5]:
line_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_line.unique())]:
    line_mapping[k] = v

df["an_line_no"] = df.an_line.map(line_mapping)  

In [6]:
cell_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_cell.unique())]:
    cell_mapping[k] = v

df["an_cell_no"] = df.an_cell.map(cell_mapping)  

In [7]:
robot_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_robot.unique())]:
    robot_mapping[k] = v

df["an_robot_no"] = df.an_robot.map(robot_mapping)  

In [8]:
message_severity_mapping = {"Error":1,"Warning":2,"Information":3}

df["message_severity"] = df.message_severity.apply(lambda x: message_severity_mapping[x] if x in message_severity_mapping.keys() else x)

In [9]:
def dfExtractMethod(df,extract_value):
    df_extract = df.loc[df['an_line_no'] == extract_value]
    return df_extract

In [10]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,no_of_messages):
    dataFrame = pd.DataFrame()
    
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
                
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
        
        dataFrame = dataFrame.append(df_extract)
    return dataFrame

In [11]:
df_ErrorsPerLinesPattern = dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,10)
df_ErrorsPerLinesPattern.to_parquet("../data_all/event_ano_pattern_all.parquet.gzip", compression='gzip')

In [12]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
    
        df_extract.to_parquet("../data_singels/event_ano_pattern_single_{}.parquet.gzip".format(j),compression='gzip')

In [13]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,10)

In [23]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetLinesMethod(df,no_of_messages,extract_value):
    for i in range(0,28):
        df_extract = dfExtractMethod(df,i)
        df_extract = df_extract.reset_index()
        
        number1 = 0
        number2 = 0
        size = df_extract.size
        data1 = []
        
        for j in df_extract:
            data1.append(j)

        dataFrame1 = pd.DataFrame(columns=data1)
        
        for k in df_extract.values:
            if(k[3] == extract_value):
                for l in range(1,11):
                    if(number1+l <= size and number1+l >= 0):
                        data2 = df_extract.values[number1-l]
                        for index in range(number2,number2+1):
                            dataFrame1.at[index, 'event_timestamp'] = data2[0]
                            dataFrame1.at[index, 'message_number'] = data2[1]
                            dataFrame1.at[index, 'message_category'] = data2[2]
                            dataFrame1.at[index, 'message_severity'] = data2[3]
                            dataFrame1.at[index, 'an_title'] = data2[4]
                            dataFrame1.at[index, 'an_description'] = data2[5]
                            dataFrame1.at[index, 'an_line'] = data2[6]
                            dataFrame1.at[index, 'an_cell'] = data2[7]
                            dataFrame1.at[index, 'an_robot'] = data2[8]
                            dataFrame1.at[index, 'an_line_no'] = data2[9]
                            dataFrame1.at[index, 'an_cell_no'] = data2[10]
                            dataFrame1.at[index, 'an_robot_no'] = data2[11]
                            number2 = index

                    if(l == 10):
                        dataFrame1.at[number2+1, 'event_timestamp'] = k[0]
                        dataFrame1.at[number2+1, 'message_number'] = k[1]
                        dataFrame1.at[number2+1, 'message_category'] = k[2]
                        dataFrame1.at[number2+1, 'message_severity'] = k[3]
                        dataFrame1.at[number2+1, 'an_title'] = k[4]
                        dataFrame1.at[number2+1, 'an_description'] = k[5]
                        dataFrame1.at[number2+1, 'an_line'] = k[6]
                        dataFrame1.at[number2+1, 'an_cell'] = k[7]
                        dataFrame1.at[number2+1, 'an_robot'] = k[8]
                        dataFrame1.at[number2+1, 'an_line_no'] = k[9]
                        dataFrame1.at[number2+1, 'an_cell_no'] = k[10]
                        dataFrame1.at[number2+1, 'an_robot_no'] = k[11]
                        number2 = number2+1
            else:
                number1 = number1 + 1
                continue        
        
        print("Fertig{}".format(i))
        dataFrame1.to_parquet("../data_dataset/singles/event_ano_pattern_dataset_per_line_line_{}.parquet.gzip".format(i), compression='gzip')
    print("Fertig")

In [24]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetLinesMethod(df,10,1)

Fertig0
Fertig1
Fertig2
Fertig3


KeyboardInterrupt: 

In [ ]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetAllLinesMethod(df,no_of_messages):
    number2 = 0
    data1 = []
    
    df_extract = dfExtractMethod(df,i)
    df_extract = df_extract.reset_index()
    
    for j in df_extract:
        data1.append(j)

    dataFrame1 = pd.DataFrame(columns=data1)
    
    for j in range(0,28):
        df_extract = dfExtractMethod(df,i)
        df_extract = df_extract.reset_index()
        
        number1 = 0
        size = df_extract.size
               
        for k in df_extract.values:
            if(k[3] == extract_value):
                for l in range(1,11):
                    if(number1+l <= size and number1+l >= 0):
                        data2 = df_extract.values[number1-l]
                        for index in range(number2,number2+1):
                            dataFrame1.at[index, 'event_timestamp'] = data2[0]
                            dataFrame1.at[index, 'message_number'] = data2[1]
                            dataFrame1.at[index, 'message_category'] = data2[2]
                            dataFrame1.at[index, 'message_severity'] = data2[3]
                            dataFrame1.at[index, 'an_title'] = data2[4]
                            dataFrame1.at[index, 'an_description'] = data2[5]
                            dataFrame1.at[index, 'an_line'] = data2[6]
                            dataFrame1.at[index, 'an_cell'] = data2[7]
                            dataFrame1.at[index, 'an_robot'] = data2[8]
                            dataFrame1.at[index, 'an_line_no'] = data2[9]
                            dataFrame1.at[index, 'an_cell_no'] = data2[10]
                            dataFrame1.at[index, 'an_robot_no'] = data2[11]
                            number2 = index

                    if(l == 10):
                        dataFrame1.at[number2+1, 'event_timestamp'] = k[0]
                        dataFrame1.at[number2+1, 'message_number'] = k[1]
                        dataFrame1.at[number2+1, 'message_category'] = k[2]
                        dataFrame1.at[number2+1, 'message_severity'] = k[3]
                        dataFrame1.at[number2+1, 'an_title'] = k[4]
                        dataFrame1.at[number2+1, 'an_description'] = k[5]
                        dataFrame1.at[number2+1, 'an_line'] = k[6]
                        dataFrame1.at[number2+1, 'an_cell'] = k[7]
                        dataFrame1.at[number2+1, 'an_robot'] = k[8]
                        dataFrame1.at[number2+1, 'an_line_no'] = k[9]
                        dataFrame1.at[number2+1, 'an_cell_no'] = k[10]
                        dataFrame1.at[number2+1, 'an_robot_no'] = k[11]
                        number2 = number2+1
            else:
                number1 = number1 + 1
                continue        
        
    dataFrame1.to_parquet("../data_dataset/all/event_ano_pattern_dataset_per_line{}.parquet.gzip".format(i), compression='gzip')
    print("Fertig")

In [ ]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetAllLinesMethod(df,10)